## Data Processing

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import ast
import openpyxl
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

### Read Dataset

In [3]:
reddit_df = pd.DataFrame(pd.read_csv('../data/reddit_depression_dataset.csv', index_col=0))
reddit_df.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/13229182.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  reddit_df = pd.DataFrame(pd.read_csv('../data/reddit_depression_dataset.csv', index_col=0))


,subreddit,title,body,upvotes,created_utc,num_comments,label
47951,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,NaN,0.0
47952,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0
47957,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0
47959,DeepThoughts,"""I want to be like water. I want to slip throu...",NaN,25.0,1.416512e+09,2.0,0.0
47960,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0


### Data Pre-Processing

In [4]:
reddit_df.shape

(2470778, 7)

In [5]:
reddit_df.dtypes

subreddit        object
title            object
body             object
upvotes         float64
created_utc     float64
num_comments    float64
label           float64
dtype: object

In [6]:
reddit_df.columns

Index(['subreddit', 'title', 'body', 'upvotes', 'created_utc', 'num_comments',
       'label'],
      dtype='object')

Explaining the attriubutes:
- **Subreddit**: The subreddit where the post was shared
- **Title**: The title provided by the user for the post
- **Body**: The content of the post written by the user
- **Upvotes**: The number of upvotes the post received
- **Created UTC**: The time the post was created, given in UTC epoch time
- **Number of Comments**: The total count of comments on the post
- **Label**: The category assigned to the post, where 0 indicates normal and 1 indicates depressed

Checking the subreddit categories:

In [7]:
reddit_df['subreddit'].value_counts()

subreddit
teenagers       1956521
depression       290058
SuicideWatch     190364
happy             24609
DeepThoughts       9163
4                    10
6                     5
5                     5
10                    2
11                    2
8                     2
9                     2
7                     2
16                    2
30                    2
61                    1
107                   1
12                    1
1402326041            1
15                    1
31                    1
101                   1
27                    1
47                    1
Name: count, dtype: int64

According to the original dataset, there should only be the first 5 categories as subreddit categories.

In [8]:
subreddit_grouped = reddit_df.groupby('subreddit')
subreddit_grouped = subreddit_grouped.filter(lambda x: x.name not in ['teenagers', 'depression', 'SuicideWatch', 'happy', 'DeepThoughts'])
subreddit_grouped['subreddit'].value_counts()

subreddit
4             10
5              5
6              5
10             2
30             2
16             2
7              2
9              2
8              2
11             2
1402326041     1
27             1
101            1
31             1
15             1
107            1
61             1
12             1
47             1
Name: count, dtype: int64

In [9]:
subreddit_grouped.count()

subreddit       43
title           43
body            42
upvotes         42
created_utc      0
num_comments     0
label            0
dtype: int64

In [10]:
subreddit_grouped.head(43)

,subreddit,title,body,upvotes,created_utc,num_comments,label
I fucking love 50 and Em.,15,1367017722,174,0.0,NaN,NaN,NaN
Edit: Pictures of not well-known animals could be great :D,6,1376650800,43,0.0,NaN,NaN,NaN
Respect the rules please.,30,1376701449,275,0.0,NaN,NaN,NaN
Please don't give any personal information away that is obviously meant to be confidential over the internet &lt;3,5,1376835553,45,0.0,NaN,NaN,NaN
Edit: People who aren't going back to school yet can tell us if they are looking forward to it or not and tell us why.,5,1376929667,25,0.0,NaN,NaN,NaN
What do,12,1383102207,22,0.0,NaN,NaN,NaN
Then I woke up...idk it was weird.,16,1384694940,9,0.0,NaN,NaN,NaN
song I wrote for my band. Any tips or advice?,5,1385719387,6,0.0,NaN,NaN,NaN
NaN,7,1386027115,43,0.0,NaN,NaN,NaN
Add kiks too :D,4,1386219268,109,0.0,NaN,NaN,NaN


The data here has been scrambled, and is missing its numerical attributes. Thus, we will remove them from the data.

In [11]:
reddit_df.isnull().sum()

subreddit           20
title               23
body            461051
upvotes             64
created_utc        106
num_comments    113977
label              106
dtype: int64

In [12]:
# Filter the dataframe to keep only the specified subreddits
reddit_df2 = reddit_df[reddit_df['subreddit'].isin(['teenagers', 'depression', 'SuicideWatch', 'happy', 'DeepThoughts'])]

# Display the first few rows of the filtered dataframe
reddit_df2['subreddit'].value_counts()

subreddit
teenagers       1956521
depression       290058
SuicideWatch     190364
happy             24609
DeepThoughts       9163
Name: count, dtype: int64

We will not check `title` and `body` as they are ok to be empty. We will fill them with blank values later.

Checking the numerical attributes:

In [13]:
print("Upvotes range:", reddit_df2['upvotes'].min(), reddit_df2['upvotes'].max())
print(reddit_df2['label'].value_counts())

Upvotes range: 4.0 128866.0
label
0.0    1990261
1.0     480411
Name: count, dtype: int64


There is nothing out of the place here. One thing to note is the disproportionality in label groups. We will work on this more in the EDA.

Let's check the missing value count again:

In [14]:
reddit_df2.isnull().sum()

subreddit            0
title                3
body            461030
upvotes             43
created_utc         43
num_comments    113914
label               43
dtype: int64

There are still some missing values. Let's dive a bit deeper.

Checking the date value:

In [15]:
reddit_df2[reddit_df2['created_utc'].isnull()].head(43)

,subreddit,title,body,upvotes,created_utc,num_comments,label
544405,teenagers,Favorite music artist?,I'm just curious since I haven't seen any rap/...,NaN,NaN,NaN,NaN
565840,teenagers,"/r/teenagers, what would be your dream pet?",It can be any animal and it can be the usual s...,NaN,NaN,NaN,NaN
566332,teenagers,What are your beliefs /r/teenagers?,Religious or just normal beliefs will do.,NaN,NaN,NaN,NaN
566520,teenagers,My name is Snapper! What would you have liked ...,New birth name or you could say what nickname ...,NaN,NaN,NaN,NaN
566920,teenagers,How was your first day back at school?,For the ones that obviously had school today &...,NaN,NaN,NaN,NaN
592474,teenagers,Ah shit,I'm have an orthodontist appointment tomorrow ...,NaN,NaN,NaN,NaN
600347,teenagers,I just had the creepiest dream.,In my dream I was coming home from vacation. I...,NaN,NaN,NaN,NaN
606826,teenagers,What do you guys think of my song?,http://m.soundcloud.com/yellow-rockets/iron-ca...,NaN,NaN,NaN,NaN
608772,teenagers,What roles do you have in your group of friends?,For instance in my small group we have a pimp ...,NaN,NaN,NaN,NaN
610758,teenagers,Rate post,As I have not found one for the day I have cre...,NaN,NaN,NaN,NaN


`created_utc` is important to us as we will need it to perform time series analysis and visualization later. Thus, data instances without this crucial attribute should not be left included.

Just to make sure that we are not removing data instances unnecessarily:

In [16]:
missing_counts = reddit_df2[['upvotes', 'created_utc', 'label']].isnull().all(axis=1).sum()
missing_counts

43

The above shows that all rows with missing `upvotes`, `created_utc` and `label` are in the same 43 rows. Thus, we can safely remove them without causing any other issues with the data we want to keep.

In [17]:
# Remove rows with missing 'upvotes', 'created_utc', and 'label'
reddit_df3 = reddit_df2.dropna(subset=['upvotes', 'created_utc', 'label'])
reddit_df3.isnull().sum()

subreddit            0
title                3
body            461029
upvotes              0
created_utc          0
num_comments    113871
label                0
dtype: int64

For the remaining missing attributes, they are optional fields that users can input in posts. Thus, we can fill them in with blank values.

In [18]:
reddit_df3['title'].fillna('', inplace=True)
reddit_df3['body'].fillna('', inplace=True)
reddit_df3['num_comments'].fillna(0.0, inplace=True)
reddit_df3.isnull().sum()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/2564938363.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reddit_df3['title'].fillna('', inplace=True)
/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/2564938363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df3['title'].fillna('', inplace=True)

subreddit       0
title           0
body            0
upvotes         0
created_utc     0
num_comments    0
label           0
dtype: int64

### Attribute Processing:

`created_utc` is currently in a non-datetime format. Let's change it so that we can measure for time.

In [19]:
reddit_df3['date'] = pd.to_datetime(reddit_df3['created_utc'], unit='s')
reddit_df3.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/1039124288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df3['date'] = pd.to_datetime(reddit_df3['created_utc'], unit='s')


,subreddit,title,body,upvotes,created_utc,num_comments,label,date
47951,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,0.0,0.0,2014-07-14 03:35:09
47952,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0,2014-09-13 00:31:19
47957,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0,2014-11-20 04:31:58
47959,DeepThoughts,"""I want to be like water. I want to slip throu...",,25.0,1.416512e+09,2.0,0.0,2014-11-20 19:38:05
47960,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0,2014-11-20 20:36:52


Let's also reset the index.

In [20]:
reddit_df3.reset_index(inplace=True, drop=True)
reddit_df3.head()

,subreddit,title,body,upvotes,created_utc,num_comments,label,date
0,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,0.0,0.0,2014-07-14 03:35:09
1,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0,2014-09-13 00:31:19
2,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0,2014-11-20 04:31:58
3,DeepThoughts,"""I want to be like water. I want to slip throu...",,25.0,1.416512e+09,2.0,0.0,2014-11-20 19:38:05
4,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0,2014-11-20 20:36:52


Now, let's work on a difficult task: Text Processing. <br>
Currently the test is in proper language sentences, which is not very useful to a computer when performing analysis.

We start by combining the two text-heavy attributes together.

In [21]:
reddit_df3['combined_text'] = reddit_df3['title'] + ' ' + reddit_df3['body']
reddit_df3.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/2253916775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df3['combined_text'] = reddit_df3['title'] + ' ' + reddit_df3['body']


,subreddit,title,body,upvotes,created_utc,num_comments,label,date,combined_text
0,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,0.0,0.0,2014-07-14 03:35:09,Deep thoughts underdog Only when we start cons...
1,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0,2014-09-13 00:31:19,"I like this sub, there's only two posts yet I ..."
2,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0,2014-11-20 04:31:58,Rebirth! Hello. \nI am the new guy in charge h...
3,DeepThoughts,"""I want to be like water. I want to slip throu...",,25.0,1.416512e+09,2.0,0.0,2014-11-20 19:38:05,"""I want to be like water. I want to slip throu..."
4,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0,2014-11-20 20:36:52,Who am I? You could take any one cell in my bo...


So, we will process the text through tokenization, stop word removal, leminitization/stemming. <br>
We would also want to remove any non-alphanumeric characters as those may take up entity space and misclassify the text.

In [22]:
nltk.download('punkt', download_dir='./nltk_data', quiet=True)
nltk.download('punkt_tab', download_dir='./nltk_data', quiet=True)
nltk.download('stopwords', download_dir='./nltk_data', quiet=True)
nltk.download('wordnet', download_dir='./nltk_data', quiet=True)
nltk.data.path.append('./nltk_data')

In [23]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

In [24]:
reddit_df3['tokenized_text'] = reddit_df3['combined_text'].apply(lambda x: word_tokenize(x))
reddit_df3.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/1251985399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df3['tokenized_text'] = reddit_df3['combined_text'].apply(lambda x: word_tokenize(x))


,subreddit,title,body,upvotes,created_utc,num_comments,label,date,combined_text,tokenized_text
0,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,0.0,0.0,2014-07-14 03:35:09,Deep thoughts underdog Only when we start cons...,"[Deep, thoughts, underdog, Only, when, we, sta..."
1,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0,2014-09-13 00:31:19,"I like this sub, there's only two posts yet I ...","[I, like, this, sub, ,, there, 's, only, two, ..."
2,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0,2014-11-20 04:31:58,Rebirth! Hello. \nI am the new guy in charge h...,"[Rebirth, !, Hello, ., I, am, the, new, guy, i..."
3,DeepThoughts,"""I want to be like water. I want to slip throu...",,25.0,1.416512e+09,2.0,0.0,2014-11-20 19:38:05,"""I want to be like water. I want to slip throu...","[``, I, want, to, be, like, water, ., I, want,..."
4,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0,2014-11-20 20:36:52,Who am I? You could take any one cell in my bo...,"[Who, am, I, ?, You, could, take, any, one, ce..."


In [25]:
reddit_df3['stopword_removed_text'] = reddit_df3['tokenized_text'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
reddit_df3.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/3296317915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df3['stopword_removed_text'] = reddit_df3['tokenized_text'].apply(lambda x: [word for word in x if word.lower() not in stop_words])


,subreddit,title,body,upvotes,created_utc,num_comments,label,date,combined_text,tokenized_text,stopword_removed_text
0,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,0.0,0.0,2014-07-14 03:35:09,Deep thoughts underdog Only when we start cons...,"[Deep, thoughts, underdog, Only, when, we, sta...","[Deep, thoughts, underdog, start, considering,..."
1,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0,2014-09-13 00:31:19,"I like this sub, there's only two posts yet I ...","[I, like, this, sub, ,, there, 's, only, two, ...","[like, sub, ,, 's, two, posts, yet, keep, comi..."
2,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0,2014-11-20 04:31:58,Rebirth! Hello. \nI am the new guy in charge h...,"[Rebirth, !, Hello, ., I, am, the, new, guy, i...","[Rebirth, !, Hello, ., new, guy, charge, (, Be..."
3,DeepThoughts,"""I want to be like water. I want to slip throu...",,25.0,1.416512e+09,2.0,0.0,2014-11-20 19:38:05,"""I want to be like water. I want to slip throu...","[``, I, want, to, be, like, water, ., I, want,...","[``, want, like, water, ., want, slip, fingers..."
4,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0,2014-11-20 20:36:52,Who am I? You could take any one cell in my bo...,"[Who, am, I, ?, You, could, take, any, one, ce...","[?, could, take, one, cell, body, kill, ,, wou..."


In [26]:
reddit_df3['alphanum_text'] = reddit_df3['stopword_removed_text'].apply(lambda x: [word for word in x if word.isalnum()])
reddit_df3.head()

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/1863821754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df3['alphanum_text'] = reddit_df3['stopword_removed_text'].apply(lambda x: [word for word in x if word.isalnum()])


,subreddit,title,body,upvotes,created_utc,num_comments,label,date,combined_text,tokenized_text,stopword_removed_text,alphanum_text
0,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,0.0,0.0,2014-07-14 03:35:09,Deep thoughts underdog Only when we start cons...,"[Deep, thoughts, underdog, Only, when, we, sta...","[Deep, thoughts, underdog, start, considering,...","[Deep, thoughts, underdog, start, considering,..."
1,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0,2014-09-13 00:31:19,"I like this sub, there's only two posts yet I ...","[I, like, this, sub, ,, there, 's, only, two, ...","[like, sub, ,, 's, two, posts, yet, keep, comi...","[like, sub, two, posts, yet, keep, coming, bac..."
2,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0,2014-11-20 04:31:58,Rebirth! Hello. \nI am the new guy in charge h...,"[Rebirth, !, Hello, ., I, am, the, new, guy, i...","[Rebirth, !, Hello, ., new, guy, charge, (, Be...","[Rebirth, Hello, new, guy, charge, Besides, Th..."
3,DeepThoughts,"""I want to be like water. I want to slip throu...",,25.0,1.416512e+09,2.0,0.0,2014-11-20 19:38:05,"""I want to be like water. I want to slip throu...","[``, I, want, to, be, like, water, ., I, want,...","[``, want, like, water, ., want, slip, fingers...","[want, like, water, want, slip, fingers, hold,..."
4,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0,2014-11-20 20:36:52,Who am I? You could take any one cell in my bo...,"[Who, am, I, ?, You, could, take, any, one, ce...","[?, could, take, one, cell, body, kill, ,, wou...","[could, take, one, cell, body, kill, would, st..."


In [27]:
reddit_df3.shape

(2470672, 12)

So when stemming, my device hit a maximum recursion limit error. <br>
There is an option to increase this limit, but could be dangerous if someone forgets to tune it back. <br>
Thus, we will split the dataset into two and run it seperately. This way we can both reduce re-run duration and identify which half/text is causing the error.

In [28]:
stemmed_texts = []
half_df = len(reddit_df3['alphanum_text']) / 2
half_df = int(half_df)
first_half_df = reddit_df3['alphanum_text'][:half_df]
second_half_df = reddit_df3['alphanum_text'][half_df:]

In [ ]:
# for text in first_half_df:
#     stemmed_texts.append([stemmer.stem(word) for word in text])

In [45]:
# for text in second_half_df:
#     try:
#         stemmed_texts.append([stemmer.stem(word) for word in text])
#     except RecursionError:
#         print(f"RecursionError with text: {text}")

RecursionError with text: ['yeah', 'b', 'oyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy

Printed Result: RecursionError with text: ['yeah', 'b', 'oyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy']

This was the specific error created. Just by looking at it we can clearly understand why the RecursionError was caused.

In [30]:
row_with_yeah_b = reddit_df3[reddit_df3['alphanum_text'].apply(lambda x: 'yeah' in x and 'b' in x and len(x) == 3)]
row_with_yeah_b_index = row_with_yeah_b.index

print("Index of the row(s):")
print(row_with_yeah_b_index)

Index of the row(s):
Index([1658491], dtype='int64')


In [31]:
reddit_df3.loc[1658491]

subreddit                                                        teenagers
title                                                               yeah b
body                     oyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy...
upvotes                                                                5.0
created_utc                                                   1624111155.0
num_comments                                                           6.0
label                                                                  0.0
date                                                   2021-06-19 13:59:15
combined_text            yeah b oyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy...
tokenized_text           [yeah, b, oyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy...
stopword_removed_text    [yeah, b, oyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy...
alphanum_text            [yeah, b, oyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy...
Name: 1658491, dtype: object

We found our culprit.

So now we will try to remove this data instance from the dataset.

In [32]:
reddit_df3.drop(index=1658491, inplace=True)
reddit_df3.reset_index(drop=True, inplace=True)

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/2493003522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df3.drop(index=1658491, inplace=True)


So let's try the stemming from the beginning again:

In [33]:
reddit_df3['stemmed_text'] = reddit_df3['alphanum_text'].apply(lambda x: [stemmer.stem(word.lower()) for word in x])

/var/folders/1v/dvtq9m8d48z659z8qvy5rxdh0000gn/T/ipykernel_36878/4114135619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df3['stemed_text'] = reddit_df3['alphanum_text'].apply(lambda x: [stemmer.stem(word.lower()) for word in x])


Here's the final text processed data attribute.

In [34]:
reddit_df3.head()

,subreddit,title,body,upvotes,created_utc,num_comments,label,date,combined_text,tokenized_text,stopword_removed_text,alphanum_text,stemed_text
0,DeepThoughts,Deep thoughts underdog,"Only when we start considering ourselves, the ...",4.0,1.405309e+09,0.0,0.0,2014-07-14 03:35:09,Deep thoughts underdog Only when we start cons...,"[Deep, thoughts, underdog, Only, when, we, sta...","[Deep, thoughts, underdog, start, considering,...","[Deep, thoughts, underdog, start, considering,...","[deep, thought, underdog, start, consid, 99, u..."
1,DeepThoughts,"I like this sub, there's only two posts yet I ...",Anyway: Human Morality is a joke so long as th...,4.0,1.410568e+09,1.0,0.0,2014-09-13 00:31:19,"I like this sub, there's only two posts yet I ...","[I, like, this, sub, ,, there, 's, only, two, ...","[like, sub, ,, 's, two, posts, yet, keep, comi...","[like, sub, two, posts, yet, keep, coming, bac...","[like, sub, two, post, yet, keep, come, back, ..."
2,DeepThoughts,Rebirth!,Hello. \nI am the new guy in charge here (Besi...,6.0,1.416458e+09,1.0,0.0,2014-11-20 04:31:58,Rebirth! Hello. \nI am the new guy in charge h...,"[Rebirth, !, Hello, ., I, am, the, new, guy, i...","[Rebirth, !, Hello, ., new, guy, charge, (, Be...","[Rebirth, Hello, new, guy, charge, Besides, Th...","[rebirth, hello, new, guy, charg, besid, thego..."
3,DeepThoughts,"""I want to be like water. I want to slip throu...",,25.0,1.416512e+09,2.0,0.0,2014-11-20 19:38:05,"""I want to be like water. I want to slip throu...","[``, I, want, to, be, like, water, ., I, want,...","[``, want, like, water, ., want, slip, fingers...","[want, like, water, want, slip, fingers, hold,...","[want, like, water, want, slip, finger, hold, ..."
4,DeepThoughts,Who am I?,You could take any one cell in my body and kil...,5.0,1.416516e+09,4.0,0.0,2014-11-20 20:36:52,Who am I? You could take any one cell in my bo...,"[Who, am, I, ?, You, could, take, any, one, ce...","[?, could, take, one, cell, body, kill, ,, wou...","[could, take, one, cell, body, kill, would, st...","[could, take, one, cell, bodi, kill, would, st..."


### Feature Selection

Here, we would want to remove unnecessary attributes such as:
- `subreddit` is not that important as we do not need to know which sub-reddit forum the mental health post came from.
- `title` and `body` are not that useful anymore as we have text processed them to obtain `stemmed_text`.
- In that regard, `created_utc` is also less useful due to our converted `date` attribute.
- `num_comments` is also an attriubte that we want to keep to measure the frequency and correlation.
- Moving onto the mid-process text processing attributes `combined_text` and `tokenized_text` can be kept as it can be useful in context-based EDA.
- However, `stopword_removed_text` and `alphanum_text` can be omitted as we will focus on working with the final product `stemmed_text`.
- Lastly, `label` is our target label, and is definitely a must have.

Thus, we are left with `stemmed_text`, `upvotes`, `num_comments` and `label`.

In [50]:
reddit_df4 = reddit_df3[['combined_text', 'tokenized_text', 'stemmed_text', 'upvotes', 'date', 'num_comments', 'label']]
reddit_df4.head()

,combined_text,tokenized_text,stemmed_text,upvotes,date,num_comments,label
0,Deep thoughts underdog Only when we start cons...,"[Deep, thoughts, underdog, Only, when, we, sta...","[deep, thought, underdog, start, consid, 99, u...",4.0,2014-07-14 03:35:09,0.0,0.0
1,"I like this sub, there's only two posts yet I ...","[I, like, this, sub, ,, there, 's, only, two, ...","[like, sub, two, post, yet, keep, come, back, ...",4.0,2014-09-13 00:31:19,1.0,0.0
2,Rebirth! Hello. \nI am the new guy in charge h...,"[Rebirth, !, Hello, ., I, am, the, new, guy, i...","[rebirth, hello, new, guy, charg, besid, thego...",6.0,2014-11-20 04:31:58,1.0,0.0
3,"""I want to be like water. I want to slip throu...","[``, I, want, to, be, like, water, ., I, want,...","[want, like, water, want, slip, finger, hold, ...",25.0,2014-11-20 19:38:05,2.0,0.0
4,Who am I? You could take any one cell in my bo...,"[Who, am, I, ?, You, could, take, any, one, ce...","[could, take, one, cell, bodi, kill, would, st...",5.0,2014-11-20 20:36:52,4.0,0.0


To save our processing progress, we will export the latest data into a .csv file.

In [51]:
reddit_df4.to_csv('../data/reddit_depression_dataset_cleaned.csv')